In [14]:
import duckdb
import geopandas as gpd
from shapely import wkt

In [3]:
conn = duckdb.connect()

In [4]:
conn.install_extension('spatial')

In [5]:
conn.load_extension('spatial')

In [6]:
overturemaps_base = "/Volumes/PRO-G40/OvertureMaps/data/release/2025-08-20.0"
segment_path = f"{overturemaps_base}/theme=transportation/type=segment/*"

In [7]:
kc_bbox = (-0.134075,51.516082,-0.091708,51.533361)

In [9]:
(minx,miny,maxx,maxy) = kc_bbox
query = f"""
SELECT id, names.primary AS primary_name, ST_AsText(geometry) as geometry
FROM read_parquet('{segment_path}')
WHERE bbox.xmin >= {minx}
  AND bbox.xmax <= {maxx}
  AND bbox.ymin >= {miny}
  AND bbox.ymax <= {maxy}
"""

In [10]:
arrow_table = conn.execute(query).fetch_arrow_table()

In [11]:
df = arrow_table.to_pandas()

In [15]:
gdf = gpd.GeoDataFrame(
    df,
    geometry=df['geometry'].apply(wkt.loads),
    crs="EPSG:4326"
)

In [16]:
gdf.explore()